In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus
from flair.datasets import SentenceDataset
from flair.data import Token
from tqdm import tqdm
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing
from sklearn import utils
import pandas as pd
pd.set_option('max.colwidth', 0)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/mypos-ver.3.0-flair.txt .
!cp /content/drive/MyDrive/mypos-ver.3.0.txt .

# Prepare Dataset

In [5]:
dataset_file = "mypos-ver.3.0-flair.txt"

# Define the columns in dataset
columns = {0: 'text', 1: 'pos'}

# Initialize the corpus
corpus = ColumnCorpus(data_folder='.', column_format=columns, train_file=dataset_file)

2023-11-11 08:54:55,923 Reading data from .
2023-11-11 08:54:55,926 Train: mypos-ver.3.0-flair.txt
2023-11-11 08:54:55,928 Dev: None
2023-11-11 08:54:55,930 Test: None
2023-11-11 08:55:11,079 No test split found. Using 0% (i.e. 4320 samples) of the train split as test data
2023-11-11 08:55:11,113 No dev split found. Using 0% (i.e. 3888 samples) of the train split as dev data


# Custom Embeddings (word2vec)



In [6]:
df_wb = pd.read_csv("mypos-ver.3.0.txt",encoding="utf-8",names=["text"])

In [7]:
df_wb.head()

,text
0,ဒီ ဆေး က ၁၀၀ ရာခိုင်နှုန်း ဆေးဘက်ဝင် အပင် များ မှ ဖော်စပ် ထား တာ ဖြစ် တယ် ။
1,အသစ် ဝယ် ထား တဲ့ ဆွယ်တာ က အသီး ထ နေ ပါ ပေါ့ ။
2,မ ကျန်းမာ လျှင် နတ်ဆရာ ထံ မေးမြန်း ၍ သက်ဆိုင်ရာ နတ် တို့ အား ပူဇော်ပသ ရ သည် ။
3,ပေဟိုင်ဥယျာဉ် ။
4,နဝမ အိပ်မက် ကောသလမင်းအိပ်မက် ၉ နက်ရှိုင်း ကျယ်ဝန်း သော ရေကန် ကြီး တစ် ခု တွင် သတ္တဝါ တို့ ဆင်း ၍ ရေသောက် ကြ ၏ ။


In [8]:
def tag_doc(df,label):
    result = []
    prefix = label
    for i, t in zip(df.index, df):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [9]:
tagged_docs = tag_doc(df_wb["text"],"doc")

In [10]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, vector_size=100, negative=5, window=5, workers=cores, alpha=0.065, min_alpha=0.001)
model_ug_cbow.build_vocab([x.words for x in tqdm(tagged_docs)])

100%|██████████| 43072/43072 [00:00<00:00, 378878.16it/s]


In [11]:
%%time
for epoch in range(10):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(tagged_docs)]), total_examples=len(tagged_docs), epochs=1)
    model_ug_cbow.alpha -= 0.004
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 43072/43072 [00:00<00:00, 1989768.62it/s]


CPU times: user 9.65 s, sys: 56.6 ms, total: 9.71 s
Wall time: 6.47 s


In [12]:
model_ug_cbow.save("word2vec.model")

In [13]:
from flair.embeddings import TokenEmbeddings
from gensim.models import Word2Vec
from flair.data import Sentence
from typing import List, Dict, Any
import numpy as np
import torch

class CustomWordEmbeddings(TokenEmbeddings):
    embeddings_name = "custom_word_embeddings"
    def __init__(self, model_path):
        super().__init__()
        self.name = "custom_word_embeddings"
        self.static_embeddings = True
        self.model_path = model_path
        self.model = Word2Vec.load(model_path)

    @property
    def embedding_length(self):
        return self.model.vector_size

    def embed(self, sentences: List[Sentence]) -> List[Sentence]:
        for sentence in sentences:
            for token in sentence.tokens:
                if token.text in self.model.wv:
                    embedding = np.array(self.model.wv[token.text])
                    embedding_tensor = torch.tensor(embedding)  # Convert to a PyTorch tensor
                    token.set_embedding(self.name, embedding_tensor)
                else:
                    # Handle out-of-vocabulary words
                    token.set_embedding(self.name, torch.zeros(self.model.vector_size))
        return sentences

    @classmethod
    def from_params(cls, params: Dict[str, Any]) -> "CustomWordEmbeddings":
        model_path = params.get("model_path")
        return cls(model_path)

    def to_params(self) -> Dict[str, Any]:
        return {"model_path": self.model_path}


In [14]:
from flair.embeddings.base import EMBEDDING_CLASSES
model_path = "word2vec.model"
# Create a custom TokenEmbeddings object
custom_word_embeddings = CustomWordEmbeddings(model_path)

EMBEDDING_CLASSES.update({
    "custom_word_embeddings": custom_word_embeddings
})

In [15]:
## test
from flair.data import Sentence

sentence1 = Sentence("ဟဲလို တင်းမ် မင်္ဂလာ ပါ")
sentence2 = Sentence("ဒီနေ့ မီတင် ရှိ မယ် နော်")

# Embed sentences using custom embedding
sentences = [sentence1, sentence2]
embedded_sentences = custom_word_embeddings.embed(sentences)

# Print embedded vectors for tokens in each sentence
for sentence in embedded_sentences:
    for token in sentence.tokens:
        print(f"Token: {token.text}, Embedding: {token.get_embedding().numpy()}")

Token: ဟဲလို, Embedding: [ 0.3368485  -0.05676079  0.9369861   0.33802754 -0.31986502 -0.42675695
  0.48137224 -0.15367013 -0.23659807 -0.36097813 -0.03043462  0.9305418
 -0.5686276   0.5049638   0.11482988 -0.2776011   0.6050444   0.30179742
  0.21278134 -0.56342727 -0.2058191   0.12706596  0.76756513 -0.10850968
 -0.5881374  -0.04279719 -0.557096   -0.08774576  0.335045    0.14845648
  0.31835693  0.33175445 -0.20714585 -0.25352716  0.09300841  0.19685848
 -0.38675672 -0.12135267 -0.06040484 -0.35751283  0.36362192 -0.3338699
  0.49545383 -0.20312753  0.72002167 -0.40316752 -0.9428144  -0.0254768
  0.7125956  -0.58786684 -0.45684573 -0.5788844   0.5501498  -0.18747619
 -0.30516037  0.13785106  0.04625937 -0.35307497  0.01141917 -0.39901528
  0.7602262  -0.14059404  0.4028461   0.09337012  0.80735284  0.49872413
 -0.14325088 -0.44681272 -0.8533135   0.2440351  -0.5069529   0.05412304
 -0.05622599  0.09167283  0.64084214  0.09122819  0.02850553  0.42168605
  0.2055734  -0.0514631  -0.3

# Sequence Tagging

In [17]:
label_type = 'pos'

# Create a label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)

# Create model
model = SequenceTagger(hidden_size=256,
                      embeddings=custom_word_embeddings,
                      tag_dictionary=label_dict,
                      tag_type=label_type)

# Create the trainer and train the model
trainer = ModelTrainer(model, corpus)
trainer.train('pos_tagger', learning_rate=0.1, mini_batch_size=32, max_epochs=10)

2023-11-11 08:56:00,832 Computing label dictionary. Progress:


0it [00:00, ?it/s]
34989it [00:01, 20409.41it/s]

2023-11-11 08:56:02,567 Dictionary created for label 'pos' with 16 values: part (seen 108156 times), n (seen 84881 times), ppm (seen 70070 times), v (seen 62529 times), punc (seen 43869 times), pron (seen 16531 times), conj (seen 14349 times), adj (seen 12758 times), adv (seen 8708 times), num (seen 4807 times), tn (seen 4712 times), fw (seen 2517 times), int (seen 534 times), abb (seen 288 times), sb (seen 225 times), O (seen 1 times)
2023-11-11 08:56:02,569 SequenceTagger predicts: Dictionary with 16 tags: part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O


2023-11-11 08:56:02,586 ----------------------------------------------------------------------------------------------------
2023-11-11 08:56:02,590 Model: "SequenceTagger(
  (embeddings): CustomWordEmbeddings()
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=18, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-11-11 08:56:02,593 ----------------------------------------------------------------------------------------------------
2023-11-11 08:56:02,596 Corpus: 34989 train + 3888 dev + 4320 test sentences
2023-11-11 08:56:02,599 ----------------------------------------------------------------------------------------------------
2023-11-11 08:56:02,600 Train:  34989 sentences
2023-11-11 08:56:02,604         (train_with_dev=False, train_with_test=False)
2023-11-

100%|██████████| 61/61 [00:15<00:00,  3.99it/s]


...

100%|██████████| 61/61 [00:20<00:00,  3.03it/s]


2023-11-11 09:37:59,228 DEV : loss 0.20708931982517242 - f1-score (micro avg)  0.9358
2023-11-11 09:37:59,396  - 0 epochs without improvement
2023-11-11 09:37:59,398 saving best model
2023-11-11 09:37:59,415 ----------------------------------------------------------------------------------------------------
2023-11-11 09:37:59,418 Loading model from best epoch ...
2023-11-11 09:37:59,476 SequenceTagger predicts: Dictionary with 18 tags: part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O, <START>, <STOP>


100%|██████████| 68/68 [00:12<00:00,  5.27it/s]


2023-11-11 09:38:13,249 
Results:
- F-score (micro) 0.9361
- F-score (macro) 0.8363
- Accuracy 0.9361

By class:
              precision    recall  f1-score   support

        part     0.9632    0.9303    0.9465     13394
           n     0.8705    0.9540    0.9103     10343
         ppm     0.9705    0.9864    0.9784      8510
           v     0.9387    0.9269    0.9328      7597
        punc     0.9968    0.9981    0.9975      5357
        pron     0.9644    0.9592    0.9618      2036
        conj     0.8924    0.9323    0.9119      1744
         adj     0.8390    0.7516    0.7929      1546
         adv     0.8863    0.7032    0.7842      1031
         num     0.8944    0.9202    0.9071       589
          tn     0.9641    0.9164    0.9396       586
          fw     0.7306    0.3422    0.4661       412
         int     0.9091    0.8108    0.8571        74
         abb     1.0000    0.3556    0.5246        45
          sb     0.8667    0.5000    0.6341        26

    accuracy         

{'test_score': 0.9361231000187652}

# Predict

In [18]:
# load the model you trained
model = SequenceTagger.load('pos_tagger/final-model.pt')

# create example sentence
sentence = Sentence('မြန်မာနိုင်ငံ မှာ မောင်မောင် နဲ့ ကျွန်တော် ပဲ စာမေးပွဲ အောင် တာ ။')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2023-11-11 09:38:21,977 SequenceTagger predicts: Dictionary with 18 tags: part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O, <START>, <STOP>
Sentence[10]: "မြန်မာနိုင်ငံ မှာ မောင်မောင် နဲ့ ကျွန်တော် ပဲ စာမေးပွဲ အောင် တာ ။" → ["မြန်မာနိုင်ငံ"/n, "မှာ"/ppm, "မောင်မောင်"/n, "နဲ့"/ppm, "ကျွန်တော်"/pron, "ပဲ"/part, "စာမေးပွဲ"/v, "အောင်"/part, "တာ"/part, "။"/punc]


In [19]:
# create example sentence
sentence = Sentence('ထို အစီအစဉ် မှာ တိုကျို တွင် အုပ်ချုပ်ရေး ဆိုင်ရာ ပညာရပ် များ ကို လေ့လာ နေ သော သခင်ထွန်းအုပ် အား ဟိုက်နန် တွင် စစ်ပညာ များ သင်ကြား နေ သည့် သခင်အောင်ဆန်း နှင့် တွေ့ဆုံ ၍ သခင်လူငယ် နှစ် ဖွဲ့ စည်းလုံးညီညွတ်ရေး ဆွေးနွေး ကြ ရန် ဖြစ် သည် ။')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

Sentence[36]: "ထို အစီအစဉ် မှာ တိုကျို တွင် အုပ်ချုပ်ရေး ဆိုင်ရာ ပညာရပ် များ ကို လေ့လာ နေ သော သခင်ထွန်းအုပ် အား ဟိုက်နန် တွင် စစ်ပညာ များ သင်ကြား နေ သည့် သခင်အောင်ဆန်း နှင့် တွေ့ဆုံ ၍ သခင်လူငယ် နှစ် ဖွဲ့ စည်းလုံးညီညွတ်ရေး ဆွေးနွေး ကြ ရန် ဖြစ် သည် ။" → ["ထို"/adj, "အစီအစဉ်"/n, "မှာ"/ppm, "တိုကျို"/n, "တွင်"/ppm, "အုပ်ချုပ်ရေး"/part, "ဆိုင်ရာ"/n, "ပညာရပ်"/n, "များ"/part, "ကို"/ppm, "လေ့လာ"/v, "နေ"/part, "သော"/part, "သခင်ထွန်းအုပ်"/n, "အား"/ppm, "ဟိုက်နန်"/n, "တွင်"/ppm, "စစ်ပညာ"/n, "များ"/part, "သင်ကြား"/v, "နေ"/part, "သည့်"/part, "သခင်အောင်ဆန်း"/n, "နှင့်"/ppm, "တွေ့ဆုံ"/v, "၍"/conj, "သခင်လူငယ်"/n, "နှစ်"/tn, "ဖွဲ့"/v, "စည်းလုံးညီညွတ်ရေး"/part, "ဆွေးနွေး"/v, "ကြ"/part, "ရန်"/conj, "ဖြစ်"/v, "သည်"/ppm, "။"/punc]
